In [2]:
import pandas as pd
from datetime import datetime
from utils.scalability_utils import cluster_to_dataframe
from models.JGSMF_with_phases import solve_with_phases
from models.SSPMF import SSPMFModel 

In [8]:
cluster_directory = "../data/MTSP/Laporte/Tabela4/"

df = cluster_to_dataframe(cluster_directory)

df = df.sample(frac=1, random_state=817).reset_index(drop=True)

In [9]:
print(df['num_tools'].unique())
print(df['magazine_capacity'].unique())


[20 25 15]
[15 20 10  5]


In questo dataset tutti le istanze hanno 9 job e un numero di tools tra 15, 20 o 25. La capienza invece è tra 5, 10, 15 o 20.
Per l'analisi di scalabilità ho deciso di prendere tutte le combinazioni di numero di tools e capienza, per poi valutarne la velocità di esecuzione tra i 2 modelli

In [10]:
scalability_df = df.drop_duplicates(subset=['num_tools', 'magazine_capacity'])

scalability_df

,filename,num_jobs,num_tools,magazine_capacity,job_tools
0,L8-2.txt,9,20,15,"{1: [1, 2, 3, 5, 6, 7, 8, 10, 12, 13, 14, 15, ..."
1,L18-3.txt,9,25,20,"{1: [4, 9, 25], 2: [7, 15], 3: [1, 5, 11, 12, ..."
6,L31-3.txt,9,25,10,"{1: [5, 9, 14, 20, 23], 2: [12, 15, 23], 3: [1..."
13,L32-6.txt,9,25,15,"{1: [7, 14, 16, 21, 24], 2: [18, 19], 3: [2, 1..."
14,L28-6.txt,9,20,10,"{1: [10, 11, 12, 17, 20], 2: [5, 7, 8], 3: [3,..."
24,L3-1.txt,9,15,10,"{1: [6, 8, 10, 14], 2: [2, 3, 4, 6, 7, 8, 11, ..."
30,L1-6.txt,9,15,5,"{1: [7, 9, 12], 2: [2, 10], 3: [1, 5, 9, 11, 1..."
35,L30-9.txt,9,25,5,"{1: [1, 6, 13, 16, 19], 2: [3, 5, 9, 19], 3: [..."
57,L27-4.txt,9,20,5,"{1: [1, 5, 13, 15, 19], 2: [2, 5, 6, 12, 20], ..."


Ogni esperimento su queste istanze verrà ripetuto 5 volte per ottenenere la mediana del tempo di ogni combinazione. Poi verrà costruito il grafico della scalability con questi dati

In [5]:
results = []

#SSPMF Model
print("SSPMF Model")
for index,row in scalability_df.iterrows():

    print("Instance: ", row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0

    for i in range(0, 5):
        start_time = datetime.now()
        SSPMF_model = SSPMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
        SSPMF_model.optimize()
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    results.append(["SSPMF",num_jobs, num_tools, magazine_capacity, median_time])

print("-----------------------------------")
#JGSMF Model
print("JGSMF Model")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0

    for i in range(0, 5):
        start_time = datetime.now()
        job_order, switches = solve_with_phases(jobs, tools, magazine_capacity, job_tools_requirements, time_limit=3600)
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    results.append(["JGSMF",num_jobs, num_tools, magazine_capacity, median_time])

results_df = pd.DataFrame(results, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])







SSPMF Model
Instance:  L1-9.txt
Set parameter Username
Set parameter LicenseID to value 2588882
Academic license - for non-commercial use only - expires 2025-11-22
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Instance:  L24-4.txt
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Instance:  L13-6.txt
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Instance:  L17-4.txt
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Instance:  L8-10.txt
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Instance:  L2-9.txt
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Optimal solution found.
Instance:  L

In [6]:
results_df

,Model,num_jobs,num_tools,magazine_capacity,time
0,SSPMF,9,15,5,0.961002
1,SSPMF,9,25,20,0.448999
2,SSPMF,9,25,10,23.447193
3,SSPMF,9,25,15,12.244513
4,SSPMF,9,20,15,5.336124
5,SSPMF,9,15,10,0.041000
6,SSPMF,9,20,10,11.876751
7,SSPMF,9,25,5,5.867588
8,SSPMF,9,20,5,0.366002
9,JGSMF,9,15,5,2.128522


Dato che alcuni esperimenti sembrano più veloci di quanto dovrebbero (considerando le combinazioni di parametri), la cui rapidità è probabilmente dovuto ai tools richiesti da ogni job .Ora ripeterò l'esperimento eseguito sopra solo considerando 5 diverse istanze per combinazione ed eseguendone poi la mediana sulla mediana dei tempi, in modo da cercare di mitigare questo problema

In [11]:
new_scalability_df = df.groupby(['num_tools', 'magazine_capacity']).head(3)


new_scalability_df

,filename,num_jobs,num_tools,magazine_capacity,job_tools
0,L8-2.txt,9,20,15,"{1: [1, 2, 3, 5, 6, 7, 8, 10, 12, 13, 14, 15, ..."
1,L18-3.txt,9,25,20,"{1: [4, 9, 25], 2: [7, 15], 3: [1, 5, 11, 12, ..."
2,L23-1.txt,9,25,20,"{1: [1, 4, 6, 7, 11, 13, 14, 16, 18, 20, 23], ..."
3,L20-9.txt,9,25,20,"{1: [1, 2, 5, 7, 8, 9, 10, 11, 13, 14, 17, 20,..."
6,L31-3.txt,9,25,10,"{1: [5, 9, 14, 20, 23], 2: [12, 15, 23], 3: [1..."
9,L10-5.txt,9,20,15,"{1: [1, 2, 3, 5, 6, 7, 11, 14, 16, 17, 18, 20]..."
10,L29-8.txt,9,20,15,"{1: [3, 14, 17], 2: [3, 4, 12, 13, 15], 3: [2,..."
13,L32-6.txt,9,25,15,"{1: [7, 14, 16, 21, 24], 2: [18, 19], 3: [2, 1..."
14,L28-6.txt,9,20,10,"{1: [10, 11, 12, 17, 20], 2: [5, 7, 8], 3: [3,..."
15,L17-4.txt,9,25,15,"{1: [3, 4, 7, 9, 10, 17, 18, 19, 20, 22], 2: [..."


In [12]:
new_results_df = []
# JGSMF Model
print("JGSMF Model")
for (num_tools, magazine_capacity), group in new_scalability_df.groupby(['num_tools', 'magazine_capacity']):
    same_iteration_times = []
    print("Group ", num_tools, magazine_capacity)

    for index, row in group.iterrows():
        print("Instance", row['filename'])
        num_jobs = row['num_jobs']
        job_tools_requirements = row['job_tools']

        jobs = [x for x in range(1, num_jobs + 1)]
        tools = [x for x in range(1, num_tools + 1)]

        iteration_times = []

        for i in range(0, 5):
            start_time = datetime.now()
            job_order, switches = solve_with_phases(jobs, tools, magazine_capacity, job_tools_requirements, time_limit=3600)
            end_time = datetime.now()
            elapsed_time = (end_time - start_time).total_seconds()
            iteration_times.append(elapsed_time)

        median_time = pd.Series(iteration_times).median()
        same_iteration_times.append(median_time)

    overall_median_time = pd.Series(same_iteration_times).median()
    new_results_df.append(["JGSMF", num_jobs, num_tools, magazine_capacity, overall_median_time])

new_results_df = pd.DataFrame(new_results_df, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])
print("-----------------------------------")
# SSPMF Model
print("SSPMF Model")
for (num_tools, magazine_capacity), group in new_scalability_df.groupby(['num_tools', 'magazine_capacity']):
    same_iteration_times = []
    print("Group ", num_tools, magazine_capacity)

    for index, row in group.iterrows():
        print("Instance", row['filename'])
        num_jobs = row['num_jobs']
        job_tools_requirements = row['job_tools']

        jobs = [x for x in range(1, num_jobs + 1)]
        tools = [x for x in range(1, num_tools + 1)]

        iteration_times = []

        for i in range(0, 5):
            start_time = datetime.now()
            SSPMF_model = SSPMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
            SSPMF_model.optimize()
            end_time = datetime.now()
            elapsed_time = (end_time - start_time).total_seconds()
            iteration_times.append(elapsed_time)

        median_time = pd.Series(iteration_times).median()
        same_iteration_times.append(median_time)

    overall_median_time = pd.Series(same_iteration_times).median()
    new_results_df.append(["SSPMF", num_jobs, num_tools, magazine_capacity, overall_median_time])


JGSMF Model
Group  15 5
Instance L1-6.txt
Trying Phase 1 with 7 bins...
Phase 1 found a solution with 7 bins and 15 switches.
Trying Phase 2 with 8 bins...
Trying Phase 2 with 9 bins...
Found less or equal switches in phase 2 with 9 bins and 14 switches.
Checking in phase 3...
Solution found.
Trying Phase 1 with 7 bins...
Phase 1 found a solution with 7 bins and 15 switches.
Trying Phase 2 with 8 bins...
Trying Phase 2 with 9 bins...
Found less or equal switches in phase 2 with 9 bins and 14 switches.
Checking in phase 3...
Solution found.
Trying Phase 1 with 7 bins...
Phase 1 found a solution with 7 bins and 15 switches.
Trying Phase 2 with 8 bins...
Trying Phase 2 with 9 bins...
Found less or equal switches in phase 2 with 9 bins and 14 switches.
Checking in phase 3...
Solution found.
Trying Phase 1 with 7 bins...
Phase 1 found a solution with 7 bins and 15 switches.
Trying Phase 2 with 8 bins...
Trying Phase 2 with 9 bins...
Found less or equal switches in phase 2 with 9 bins and 14

GurobiError: Unable to set parameter TimeLimit to value -11.569 (minimum is 0)

In [ ]:
new_results_df